# Movie Theme Extraction and Labeling

The goal of this project is to automate the extraction of labeled themes from movie overviews. These themes can replace the overview as a faster and more consistent way of determining what the movie is about. The defined themes would also allow for faster filtering that would allow the user to find a more exact match based on viewing mood.

In [ ]:
import pandas as pd


Data used for this project can be found here https://www.kaggle.com/rounakbanik/the-movies-dataset

We will use the overviews and titles from the movies_metadata.csv

In [ ]:
df = pd.read_csv('the-movies-dataset\movies_metadata.csv')

In [ ]:
overviews = df['overview'].apply(str).tolist()
overviews = " ".join(overviews)    #prepare for word_tokenize

# Data Preparation

Names like Alan and Jack are common and will appear influence topic model distributions. Given that names are interchangeable when discussing themes, we will identify and remove them as a part of the data preparation process.

In [ ]:
import nltk

In [ ]:
tokens = nltk.word_tokenize(overviews)
tokens_pos = nltk.pos_tag(tokens)

### Named Entity Recognition:

Use named entity recognition to detect names in the corpus. The ne_chunk function is used to extract chunks from the tagged pos tokens and will return person, places and organizations. 

In [ ]:
ne_tree = nltk.ne_chunk(tokens_pos)

In [ ]:
#build list of people to remove
people = []                                 
for subtree in ne_tree.subtrees():
    if subtree.label() == 'PERSON':
        person = []
        for key, value in subtree.leaves():
            person.append(key)
        people.append(person)

In [ ]:
#counter gives count of each name 
import collections, operator
people_unique = collections.Counter(map(tuple, people))

Named entity recognition identifies some chunks that are not names. Using the frequency list of people_unique and manual check of the high frequency terms is done to create a list of misidentified entites that should remain in the corpus.

In [ ]:
exclude_list = [('Hollywood'),('Christmas','Eve'),('San', 'Francisco'), ('Las', 'Vegas'), 
                ('Santa', 'Claus'), ('Christmas',), ('Hollywood',), ('Superman',), ('Academy',), 
                ('Godzilla',), ('Academy', 'Award'), ('Buenos', 'Aires'), ('Jesus', 'Christ'),
                ('Pearl', 'Harbor'), ('Louisiana',), ('Sequel',), ('Father',), ('Wealthy',), 
                ('Disney',), ('Count', 'Dracula'),('Los', 'Angeles'), ('Monster', 'High'), 
                ('Brazil',), ('Shaolin',), ('Halloween',), ('Navy','SEALS')
               ]

In [ ]:
#return clean list of names minus the misidentified names
people_clean = []
for key,val in sorted(people_unique.items(), key=operator.itemgetter(1), reverse = True):
    if key not in exclude_list:
        people_clean.append(' '.join(key))

## Prep data:

* remove people's names
* remove stop words
* remove punctuation
* lemmatization

In [ ]:
import string
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
stops = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

other_stops = ["'s","--","..."]
stops.extend(other_stops)


texts = []
for sent in df['overview'].apply(str):
    text = nltk.word_tokenize(sent)
    text = ['PERSON' if word in people_clean else word for word in text  ]  #replace name with PERSON
    text = [word.lower() for word in text] 
    text = [word for word in text if word not in stops]
    text = [lemmatizer.lemmatize(word) for word in text]
    text = [word for word in text if word not in string.punctuation ] 
    texts.append(text)

In [ ]:
#additional cleaning 
more_stops = ["``","'ll","''"]
texts_clean = [[word for word in text if word != 'person'] for text in texts]     #remove identified names
texts_clean = [[word for word in text if word not in more_stops] for text in texts_clean]
texts_clean = [[word for word in text if len(word) > 2] for text in texts_clean]
print(texts_clean[:2])

## LDA topic modeling

Latent Dirichlet Allocation will be used for topic modeling. 

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim import corpora, models

In [ ]:
#build bag of words representation of all documents 
dictionary = corpora.Dictionary(texts_clean)
corpus = [dictionary.doc2bow(text) for text in texts_clean ]

## 300 topics chosen after multiple iterations and coherence score testing

Choosing the right number of topics is an iterative process. After multiple tests, the number of topics that gave the best results was 300. The coherence score and visualization of the topic models for human readability were part of the evaluation process.

In [ ]:
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=300, update_every=1, chunksize=5000, passes=3)

In [ ]:
#get coherence score 
from gensim.models.coherencemodel import CoherenceModel
cm = CoherenceModel(model=lda, texts=texts_clean, corpus=corpus, coherence='c_v')
coherence = cm.get_coherence()  
print(coherence)

The pyLDAvis library provides a convenient way to visualize the topics and the distribution of terms in each topic.

In [ ]:
import pyLDAvis.gensim
vis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis)

# Seed Labels

Two types of chunk patterns will form the basis of the seed phrases. We are looking to form two word phrases that represent things and terms as described by the corpus. The idea is that themes would be explicitly described during some of the overview creations and that similarity mesures can be used to aplly the common themes as labels.

In [ ]:
thing_chunker = "CHUNK: {<NN><NN>}"
term_chunker = "CHUNK: {<JJ.*><NN>}"
thing_parser = nltk.RegexpParser(thing_chunker)
term_parser = nltk.RegexpParser(term_chunker)

In [ ]:
thing_tree = thing_parser.parse(tokens_pos)
term_tree = term_parser.parse(tokens_pos)

In [ ]:
labels = []
for subtree in thing_tree.subtrees():
    if subtree.label() == 'CHUNK':
        label = []
        for key, value in subtree.leaves():
            label.append(key)
        #print(label)
        labels.append(label)
for subtree in term_tree.subtrees():
    if subtree.label() == 'CHUNK':
        label = []
        for key, value in subtree.leaves():
            label.append(key)
        #print(label)
        labels.append(label)

In [ ]:
unique_labels = [list(label) for label in set(tuple(label) for label in labels)]

We can examine the seed list to find potential matches that contain a specific word.

In [ ]:
potentials = [label for label in unique_labels if 'club' in label]
potentials

### High potential phrases

Here we will collect the top high potential phrases for each topic. 
1. For each word in the topic model find potentials that have one matching word
2. Convert each matching phrase to a bow model using the same dictionary used for lda training
3. Get topic model for the phrase
4.  Save if topic matches current topic and probability is high 

In [ ]:
top_matches = {}
for topic_id, words in lda.print_topics(300):
    top_match = []
    top = 0
    for word, rat in lda.show_topic(topic_id):
        potentials = [label for label in unique_labels if word in label]
        for potential in potentials:
            unseen = dictionary.doc2bow(potential)
            for topic, prob in lda[unseen]:
                if topic ==topic_id and prob > 0.6:
                    if topic in top_matches:
                        top_matches[topic].append(list(potential))
                    else:
                        top_matches.update({topic : [list(potential)]})
                    if prob > top:
                        top = prob
                        top_match = list(potential)
    print('Top Match:',topic_id, top_match)

# Replace overview with a list of themes

In [ ]:
#create Movie class
class Movie:
    def __init__(self, doc_id, title, overview, topics, labels, label_score):
        self.doc_id = doc_id
        self.title = title
        self.overview = overview
        self.topics = topics
        self.labels = dict(labels)
        self.label_score = label_score

In [ ]:
#tfidf model to be used to improve similarity scores
tfidf = models.TfidfModel(corpus)

## Get a single label for each topic in an overview

1. Get all the topics  above a minimum threshold for each document
2. Get all the high potential phrases for the topic
3. Covert the phrase to a tfidf bow model
4. Compare phrase and document cosine similarity 
5. Keep highest score above a minimum threshold

If no label meets the minimum threshold then we ignore that topic for overview replacement.

In [ ]:
from gensim import matutils


def get_movie_labels(doc_id):  
    top_pot_labels = {}
    for topic_id,prob in lda.get_document_topics(corpus[doc_id], minimum_probability=0.03): 
        top_pot_score = 0
        top_pot_label = []
        for pot in top_matches[topic_id]:
            pot_corpus = tfidf[dictionary.doc2bow(pot)]
            #label score
            pot_score = matutils.cossim(tfidf[corpus[doc_id]], pot_corpus)
            if pot_score > top_pot_score and pot_score > 0.05:
                top_pot_label = list(pot)
                top_pot_score = pot_score
        if len(top_pot_label) > 0 : top_pot_labels.update({topic_id : list(top_pot_label)})

    
    top_pot_sent = []
    for key in top_pot_labels:
        top_pot_sent.append(list(top_pot_labels[key]))
    top_pot_sent = [word for label in top_pot_sent for word in label]

    label_score = matutils.cossim(tfidf[corpus[doc_id]], tfidf[dictionary.doc2bow(top_pot_sent)])
    title = df['title'].loc[doc_id]
    overview = df['overview'].loc[doc_id ]
    topics = {}
    for key, value in lda[corpus[doc_id]]:
        topics.update({key : value})
    
    new_movie = Movie(doc_id, title, overview, topics, top_pot_labels, label_score )
    return new_movie

## Build dictionary of movie objects with theme lists and write to csv file

In [ ]:
movies = {}
for doc_id in range(len(corpus)):
    movies.update( {doc_id: get_movie_labels(doc_id)})

In [ ]:
import csv

with open('movie_labels.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['doc_id', 'title','overview', 'topics', 'labels','label_score' ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    for movie in range(len(movies)):
        try:
            writer.writerow({'doc_id': movies[movie].doc_id, 'title':movies[movie].title, 'overview':movies[movie].overview ,
                         'topics': movies[movie].topics, 'labels': movies[movie].labels , 'label_score': movies[movie].label_score})
        except:
            print(movie, 'failed')

## Build dictionary of topics and their associated documents

Gather the list of documents for each topic. The labels from the movie object is used instead of the full topic distribution. This will increase the accuracy of the filtered list of movies that match multiple topics. 

In [ ]:
from collections import defaultdict
topics_docs = defaultdict(list)

for i, doc in enumerate(movies):
    for key in movies[i].labels:
        topics_docs[key].append(i)

## Get global label based on frequency of words in associated documents



1. Get documents associated with topic 
2. Build word frequency list for top x words
3. Covert the phrase to a tfidf bow model
4. Get cosine similarity for high potential phrases and top frequency list
5. Keep the highest score as the top label



In [ ]:
def get_top_label(topic_id):
    num_words  = 10       #number of top frequency words to return
    prob_thres = 0.1      #minimum threshold for probability of topic in document
    topic_docs = []

    for doc in topics_docs[topic_id]:
        doc_topics = lda.get_document_topics(corpus[doc], minimum_probability=prob_thres)
        for key, value in doc_topics:
            if  key == topic_id:
                topic_docs.append(corpus[doc])

    freq_dict = {}
    for doc in topic_docs:
        for word, count in doc:
            if word in freq_dict:
                freq_dict[word] += count
            else:
                freq_dict.update({word: count})

    sorted_dict =  sorted(freq_dict.items(), key=operator.itemgetter(1), reverse=True)


    freq_words = []
    for key, value in sorted_dict[:num_words]:
        freq_words.append(dictionary.id2token[key])
    print('Word Frequency:',freq_words)
    freq_corpus = tfidf[dictionary.doc2bow(freq_words)]

    top_score = 0
    top_score_match = []

    for pot in top_matches[topic_id]:
        pot_corpus = tfidf[dictionary.doc2bow(pot)]
        #label score
        pot_score = matutils.cossim(freq_corpus , pot_corpus)
        if pot_score > top_score:
            top_score_match = list(pot)
            top_score = pot_score
            
    return top_score_match




============================


# Demo Section

============================

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

def print_movie_details(movie_id): 
    temp_movie = get_movie_labels(movie_id ) 
    print("Title:", temp_movie.title)
    print("Overview:" ,temp_movie.overview)
    pp.pprint(temp_movie.labels)

## Sample document 

use: <br>
print_movie_details(movie_id)<br>
movie_id range from 0 - 45465


In [ ]:
print_movie_details(1256)

## Browse topics

topic_id ranges from 0 - 299 <br>
get_top_label(topic_id)

In [ ]:
browse_topic_id =  189
print('LDA distribution:', lda.show_topic(browse_topic_id))
print('Theme:',get_top_label(browse_topic_id))

## Find intersection of document lists for multiple topics to filter down to a small list movies

topic_id ranges from 0 - 299 <br> 
chain set(topics_docs[topic_id]) together to find the intersection of documents for topics chained

In [ ]:
for doc_id in list(set(topics_docs[45]) & set(topics_docs[189]) ):
    print_movie_details(doc_id)
    print("=======================================")